In [1]:
# parquet-fromcsv -s core_r10m_c20_schema.txt -i core_r10m_c20.csv -o core_r10m_c20.parquet
# /mnt/arrow-rs/target/release/examples/filterscan core_r10m_c20.parquet 4 int point 511 0
# python3 ../../benchmark/generator_v2/filter_generator.py core_r10m_c20.csv core_r10m_c20_config.json ../../benchmark/generator_v2/filter_config/core_filter.yaml
# cargo build --package parquet --example filterscan --features="arrow async" --release 

## parquet cpp
# /mnt/arrow-private/cpp/out/build/openformat-release/release/filter_scan core_r10m_c20.parquet --batch-size=1024 --columns=18 --filter_idx=18 --filter_type=1 --type=2 --v1=0000rg67SI8 --v2=5e7WzM1Ny9LMiGrx4rZBPVDih0
# import duckdb
# con = duckdb.connect(database=':memory:')
# con.execute('''select count(*) from 'lineitem_1M_shipdate_bf.parquet' 
# where l_orderkey=22142214''').fetchall()
# con.execute('''select count(*) from 'lineitem_1M_shipdate_bf.parquet' 
# group by l_orderkey''').fetchall()

In [19]:
import os
import sys
import datetime
import pathlib
import pandas as pd
import pyarrow.orc as po
dir_path = pathlib.Path(os.path.abspath('')).parent.resolve()
HOME_DIR = str(dir_path).split('/OpenFormat')[0]

timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

PROJ_SRC_DIR = f'{HOME_DIR}/OpenFormat'
sys.path.insert(1, f'{PROJ_SRC_DIR}')
from python.scripts.utils import *

In [22]:
%time
# generate csv
# file_base = 'lineitem_1M_shipdate'
file_base = 'lineitem_1M'
import pyarrow.parquet as pq
parquet_file = pq.ParquetFile(f'{file_base}.parquet')
table = parquet_file.read()
from pyarrow import csv
# csv.write_csv(table, f'{file_base}.csv')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 12.4 µs


In [4]:
# generate parquet and orc
# os.chdir(f'{PROJ_SRC_DIR}/python')
# os.system(f'python3 scripts/orc_exp.py {file_base} orc_default')
orc_config = {'compression':'SNAPPY', 'compression_strategy':'speed', 
              'dictionary_key_size_threshold': 0.8}
os.chdir(f'{PROJ_SRC_DIR}/python/index')
os.system(f'parquet-fromcsv -s {file_base}_arrow_schema.txt -i \
    {file_base}.csv -o {file_base}.parquet --has-header')
po.write_table(
        table, f'{file_base}.orc', **orc_config)
# orc_config = enumerate_config(f'{PROJ_SRC_DIR}/python/experiments/orc_default.json')
# orc_name_suffix = '_' + "_".join([str(i) for i in list(orc_config[0].values())]) + '.orc'
# os.system(f'mv ../{file_base}{orc_name_suffix} ./{file_base}.orc')

In [23]:
%%time
# generate parquet and orc files for diff fpp
fpp_list = [0.0001, 0.001, 0.01, 0.05, 0.1, 0.2]
col_name = 'l_partkey'
col_idx = 1
proj_type = 'one'
orc_config['bloom_filter_columns'] = [col_idx+1]
for fpp in fpp_list:
    os.system(f'parquet-fromcsv -s {file_base}_arrow_schema.txt -i \
        {file_base}.csv -o {file_base}_fpp{fpp}.parquet --has-header \
        --fpp {fpp} --bf-col {col_name}')
    orc_config['bloom_filter_fpp'] = fpp
    po.write_table(
        table, f'{file_base}_fpp{fpp}.orc', **orc_config)

CPU times: user 11min 8s, sys: 11.7 s, total: 11min 20s
Wall time: 22min 5s


In [21]:
os.system('rm outputs/stats.json')
output_stats = {}
proj_type = 'one'
target_key = 1284483
for i in range(3):
    for fmt in ['parquet', 'orc']:
        for fpp in fpp_list:
            os.system('sync; echo 3 > /proc/sys/vm/drop_caches')
            if fmt == 'parquet':
                cmd = f"parquet-bf -f {file_base}_fpp{fpp}.parquet \
                    -i {col_idx} -l {target_key} -r {target_key}"
                output_stats['time'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
                output_stats['time_preload'] = os.popen(cmd).read().split('\n')[0].split(' ')[-2]
            else:
                cmd = f"{HOME_DIR}/orc/build/c++/test/FilterExp \
                    {file_base}_fpp{fpp}.orc {col_name} int point \
                    {target_key} {target_key} {proj_type}"
                output_stats['time'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
                output_stats['time_preload'] = float(os.popen(cmd).read().split('\n')[0].split(' ')[-1])*1000
            output_stats['format'] = fmt
            output_stats['i'] = i
            output_stats['fpp'] = fpp
            output_stats['extra_size'] = os.path.getsize(f'{file_base}_fpp{fpp}.{fmt}') - \
                os.path.getsize(f'{file_base}.{fmt}')
            parse_output(output_stats)
collect_results()
os.system('mv outputs/stats.csv ../outputs/{}_{}.csv'.format('bf_', timestamp))

0